# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235315758145                   -0.50    7.0
  2   -7.250361753122       -1.82       -1.41    1.0
  3   -7.251208906266       -3.07       -2.26    1.0
  4   -7.251313871375       -3.98       -2.37    2.0
  5   -7.251332698634       -4.73       -2.73    1.0
  6   -7.251338037360       -5.27       -3.19    2.0
  7   -7.251338728270       -6.16       -3.68    1.0
  8   -7.251338784520       -7.25       -3.95    2.0
  9   -7.251338797311       -7.89       -4.43    1.0
 10   -7.251338798515       -8.92       -5.04    2.0
 11   -7.251338798668       -9.82       -5.25    2.0
 12   -7.251338798696      -10.56       -5.53    1.0
 13   -7.251338798704      -11.10       -6.10    2.0
 14   -7.251338798704      -12.14       -6.66    1.0
 15   -7.251338798705      -13.19       -7.05    2.0
 16   -7.251338798705      -14.35       -7.60    1.0
 17   -7.251338798705      -14.45       -7.98 

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06877308536216521
[ Info: Arnoldi iteration step 2: normres = 0.5203055277771127
[ Info: Arnoldi iteration step 3: normres = 0.9411504445307582
[ Info: Arnoldi iteration step 4: normres = 0.28450240612734384
[ Info: Arnoldi iteration step 5: normres = 0.4495010349525038
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.70e-02, 5.69e-02, 2.83e-01, 3.39e-01, 5.27e-02)
[ Info: Arnoldi iteration step 6: normres = 0.35607507896654544
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.19e-02, 2.20e-01, 2.56e-01, 8.12e-02, 5.23e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0842798602735927
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.14e-04, 2.16e-02, 1.03e-02, 3.40e-02, 6.83e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10195001355557103
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.29e-